In [0]:
from pyspark import *
from pyspark.sql.functions import col, current_date

In [0]:
df=spark.read.csv("/Volumes/cgi_dev/naval/formula1/circuits.csv",header=True,inferSchema=True)

df_final=df.withColumn("ingestion_date",current_date()).withColumn("metadata",col("_metadata"))

df_final.write.mode("overwrite").saveAsTable("cgi_dev.reetika_bronze.circuits")

In [0]:
%sql
create or replace table cgi_dev.reetika_bronze.constructors as
(select *,current_date as ingestion_date, _metadata as metadata from json.`/Volumes/cgi_dev/naval/formula1/constructors.json`)

In [0]:
df=spark.read.option("multiline",True).json("/Volumes/cgi_dev/naval/formula1/pit_stops.json")

df_final=df.withColumn("ingestion_date",current_date()).withColumn("metadata",col("_metadata"))

df_final.write.mode("overwrite").saveAsTable("cgi_dev.reetika_bronze.pit_stops")

In [0]:
%sql
create or replace table cgi_dev.reetika_bronze.drivers as
(select *,current_date as ingestion_date, _metadata as metadata from json.`/Volumes/cgi_dev/naval/formula1/drivers.json`)

In [0]:
df=spark.table("cgi_dev.reetika_bronze.drivers")

In [0]:
df_final=df\
.withColumn("forename",col("name.forename"))\
.withColumn("surname",col("name.surname"))\
.drop("name","metadata","ingestion_date")

In [0]:
df_final.write.saveAsTable("cgi_dev.reetika_silver.drivers")

In [0]:
%sql
select *, name.forename, name.surname from cgi_dev.reetika_bronze.drivers